# Challenge: Promotions

In this challenge, you'll develop codes to parse and analyze data returned from another API on Zalando such as [Promos homme (Men's Promotions)
](https://www.zalando.fr/promo-homme/) or [Promos femme (Women's Promotions)](https://www.zalando.fr/promo-femme/). The workflow is almost the same as in the guided lesson but you'll work with different data.

## Obtaining the link

Wrote your codes in the cell below to obtain the data from the API endpoint you choose. A recap of the workflow:

1. Examine the webpages and choose one that you want to work with.

1. Use Google Chrome's DevTools to inspect the XHR network requests. Find out the API endpoint that serves data to the webpage.

1. Test the API endpoint in the browser to verify its data.

1. Change the page number offset of the API URL to test if it's working.

In [20]:
import json
import requests
import pandas as pd
from urllib.request import urlopen
from pandas.io.json import json_normalize
url='https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset=84'

response = urlopen(url)
results = json.load(response)
results


flattened_data = json_normalize(results)

flattened_data1 = json_normalize(flattened_data.articles[0])
flattened_data1

<ipython-input-20-0d7d42a05e6b>:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  flattened_data = json_normalize(results)
<ipython-input-20-0d7d42a05e6b>:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  flattened_data1 = json_normalize(flattened_data.articles[0])


,sku,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,delivery_promises,price.original,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,amount,outfits
0,PO222O05E-Q12,Sweat à capuche - black/red,"[XS, S, M, L, XL, XXL]",polo-ralph-lauren-t-shirt-a-manches-longues-bl...,[{'path': 'spp-media-p1/96961807be6c34ffaf4530...,Polo Ralph Lauren,True,[],"[{'key': 'discountRate', 'value': '-15%', 'tra...",clothing,[],"79,95 €","67,95 €",False,False,False,False,NaN,NaN
1,PI922P030-C11,Polo - mottled light grey,"[S, M, L, XL, XXL]",pier-one-polo-mottled-light-grey-pi922p030-c11,[{'path': 'spp-media-p1/f623b25cadbf36f19f955d...,Pier One,False,[],"[{'key': 'discountRate', 'value': '-15%', 'tra...",clothing,[],"14,99 €","12,79 €",False,False,False,False,NaN,NaN
2,PI922O04L-K11,T-shirt imprimé - dark blue,"[XS, S, M, L, XL, XXL, 4XL, 5XL, 6XL]",pier-one-t-shirt-imprime-dark-blue-pi922o04l-k11,[{'path': 'spp-media-p1/a9b7cd1f667f351096fb08...,Pier One,False,[],"[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,[],"14,99 €","12,04 €",False,False,False,False,NaN,NaN
3,PI912K04W-Q11,Bottines - black,"[40, 41, 42, 43, 44, 45, 46, 47, 48]",pier-one-bottines-black-pi912k04w-q11,[{'path': 'spp-media-p1/5c1dd6fb2e323f43bcd390...,Pier One,False,[],"[{'key': 'discountRate', 'value': '-50%', 'tra...",shoe,[],"39,99 €","19,99 €",False,False,False,False,NaN,NaN
4,NE344A07T-502,NEW YORK YANKEES - Casquette - navy/white,[One Size],new-era-new-york-yankees-casquette-bleu-ne344a...,[{'path': 'spp-media-p1/09b1b34737643da7a415ac...,New Era,False,[],"[{'key': 'discountRate', 'value': '-25%', 'tra...",accessoires,[],"19,95 €","14,95 €",False,False,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,6CA22O022-A11,T-shirt imprimé - white,"[XS, S, M, L, XL, XXL]",calvin-klein-t-shirt-imprime-white-6ca22o022-a11,[{'path': 'spp-media-p1/02684510f7143cc7823ad7...,Calvin Klein,False,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",clothing,[],"49,95 €","34,95 €",False,False,False,False,NaN,NaN
80,C7642E03H-C11,BERMUDA - Short de sport - grey,"[S, M, L, XL, XXL]",champion-big-logo-bermuda-short-de-sport-c7642...,[{'path': 'spp-media-p1/f17fbd134bca33c8b8c707...,Champion,False,[],"[{'key': 'discountRate', 'value': '-10%', 'tra...",clothing,[],"29,95 €","26,95 €",False,False,False,False,NaN,NaN
81,PI912M0AB-K11,Chaussures bateau - dark blue,"[40, 41, 42, 43, 44, 45, 46, 47, 48]",pier-one-chaussures-bateau-dark-blue-pi912m0ab...,[{'path': 'spp-media-p1/ea3b3d7c6fb63798a1a06f...,Pier One,False,[],"[{'key': 'discountRate', 'value': '-20%', 'tra...",shoe,[],"29,99 €","24,09 €",False,False,False,False,NaN,NaN
82,AD121001R-Q11,STRIPES CREW UNISEX - Sweatshirt - black,"[XS, S, M, L, XL, XXL]",adidas-originals-3-stripes-crew-unisex-sweatsh...,[{'path': 'spp-media-p1/a3b880c4c63f4a96a0fdcd...,adidas Originals,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -10...",clothing,[],"49,95 €","44,95 €",True,False,True,False,NaN,NaN


## Reading the data

In the next cell, use Python to obtain data from the API endpoint you chose in the previous step. Workflow:

1. Import libraries.

1. Define the initial API endpoint URL.

1. Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

1. Find out the total page count in the 1st page data.

1. Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

1. Print and review the data you obtained.

In [36]:
import json
import requests
import pandas as pd
from urllib.request import urlopen
from pandas.io.json import json_normalize
url='https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset=84'

#response = urlopen(url)
#results = json.load(response)
#results


#flattened_data = json_normalize(results)

#flattened_data1 = json_normalize(flattened_data.articles[0])
#flattened_data1

total_pages=results['pagination']['page_count']


df=pd.DataFrame()
for i in range(total_pages):
    k=84*i
    url=f'https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset={k}&sort=sale'
    response = urlopen(url)
    results = json.load(response)
    flattened_data = json_normalize(results)
    flattened_data1 = json_normalize(flattened_data.articles[0])
    flattened_data1=flattened_data1.set_index('sku')
    df = df.append(flattened_data1)

df

<ipython-input-36-ff127b23c9de>:27: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  flattened_data = json_normalize(results)
<ipython-input-36-ff127b23c9de>:28: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  flattened_data1 = json_normalize(flattened_data.articles[0])


,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,delivery_promises,...,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,outfits,amount,condition,condition_key,price.base_price
sku,,,,,,,,,,,,,,,,,,,,,
HY554B01L-Q11,Masque en tissu - black,[One Size],hype-masque-en-tissu-black-hy554b01l-q11,[{'path': 'spp-media-p1/c15457b8f18532a086b55a...,Hype,False,[],"[{'key': 'discountRate', 'value': '-70%', 'tra...",accessoires,[],...,"4,50 €",False,False,False,False,NaN,NaN,NaN,NaN,NaN
JA222T0B6-O12,JJEROCKY JACKET - Veste en similicuir - cognac,"[XS, S, M, L, XL]",jack-and-jones-jjerocky-jacket-veste-en-simili...,[{'path': 'spp-media-p1/ae896c064f6d3147872443...,Jack & Jones,False,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",clothing,[],...,"41,99 €",False,False,False,False,NaN,NaN,NaN,NaN,NaN
OS322T05O-Q11,ONSMIKE RACER - Veste en similicuir - black,"[XS, S, M, L, XL, XXL]",only-and-sons-onsmike-racer-jacket-veste-en-si...,[{'path': 'spp-media-p1/5a76970b0d383d5d9dbf64...,Only & Sons,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -50...",clothing,[],...,"19,99 €",True,False,True,False,NaN,NaN,NaN,NaN,NaN
TO122O08I-Q11,LOGO TEE - T-shirt imprimé - black,"[XS, S, M, L, XL, XXL, 3XL]",tommy-hilfiger-logo-tee-t-shirt-imprime-to122o...,[{'path': 'spp-media-p1/6a1582f6fde836a5917156...,Tommy Hilfiger,False,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",clothing,[],...,"27,95 €",False,False,False,False,NaN,NaN,NaN,NaN,NaN
JA222E0F5-K11,MARCO BOWIE - Chino - navy,"[27x30, 27x32, 28x30, 28x32, 29x30, 29x32, 29x...",jack-and-jones-marco-bowie-chino-navy-ja222e0f...,[{'path': 'spp-media-p1/33fc78cacde137fc8cecd1...,Jack & Jones,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -25...",clothing,[],...,"29,99 €",True,False,True,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C1W32G00C-S11,TONING SHOWER GEL - Gel douche - -,[],collistar-toning-shower-gel-gel-douche-c1w32g0...,[{'path': 'spp-media-p1/3b77da0a50613eadbbb859...,Collistar,False,[],"[{'key': 'discountRate', 'value': '-15%', 'tra...",beauty,[],...,"20,45 €",False,False,False,False,NaN,250 ml,NaN,NaN,"8,18 € / 100 ml"
C2342D01S-K11,TERRA VALE™ TEE - T-shirt imprimé - collegiate...,"[S, M, L, XL, XXL]",columbia-terra-vale-tee-t-shirt-de-sport-colle...,[{'path': 'spp-media-p1/79cff5854304305a9ee3f0...,Columbia,False,[],"[{'key': 'discountRate', 'value': '-10%', 'tra...",clothing,[],...,"26,95 €",False,False,False,False,NaN,NaN,NaN,NaN,NaN
BB154H011-Q11,NATHALIE - Sac banane - black,[One Size],boss-nathalie-sac-banane-black-bb154h011-q11,[{'path': 'spp-media-p1/69b8c84b7e7c3c63b920f3...,BOSS,True,[],"[{'key': 'discountRate', 'value': '-26%', 'tra...",accessoires,[],...,"260,00 €",False,False,False,False,NaN,450 g,NaN,NaN,NaN


## Bonus

Extract the following information from the data:

* The trending brand.

* The product(s) with the highest discount.

* The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

In [37]:
# your code here
#The trending brand.
print('The trending brand: '+df.brand_name.value_counts().index[0])
#The product(s) with the highest discount.

df['price.original']=df['price.original'].str.extract('(\d*,\d*)')
df['price.promotional']=df['price.promotional'].str.extract('(\d*,\d*)')

df['price.original'] = [x.replace(',', '.') for x in df['price.original']]
df['price.promotional'] = [x.replace(',', '.') for x in df['price.promotional']]
df['discount_amount']=df['price.original'].astype(float)-df['price.promotional'].astype(float)
df['discounts_of_all_goods']=df['price.promotional'].astype(float)/df['price.original'].astype(float)
df1=df.copy()
total_disc=df1.groupby(['name']).sum().discount_amount
print('The product(s) with the highest discount: '+total_disc.sort_values(ascending=False).index[0])
print('The sum of discounts of all goods: '+str(df['discounts_of_all_goods'].sum()))


The trending brand: Jack & Jones
The product(s) with the highest discount: Baskets basses - black
The sum of discounts of all goods: 26924.313794792317
